In [ ]:
import datetime

import pandas as pd
import YahooFinancials
import numpy as np

def classificator(row, location):
    if row[location] >= 1.25:
      return 4
    elif row[location] >= 0.35:
      return 3
    elif row[location] <= -1.25:
      return 0
    elif row[location] <= -0.35:
      return 1
    else:
      return 2

def historical_yahoo(start = (datetime.now() - datetime.timedelta(days=2000)).strftime("%Y-%m-%d"),
                    end= datetime.now().strftime("%Y-%m-%d"), tickers = ["BTC-USD","GOLD","AAPL",
                    "TSLA",'WFC', 'BAC', 'C' ,'EURUSD=X', 'JPY=X', 'GBPUSD=X', 'PRLAX', 'QASGX', 'HISFX']):
    ###si passem mes data de la que hauriem el model apren encara menys
    """
    tech_stocks = ['AAPL', 'MSFT', 'INTC']
    bank_stocks = ['WFC', 'BAC', 'C']
    commodity_futures = ['GC=F', 'SI=F', 'CL=F']
    cryptocurrencies = ['BTC-USD', 'ETH-USD', 'XRP-USD']
    currencies = ['EURUSD=X', 'JPY=X', 'GBPUSD=X']
    mutual_funds = ['PRLAX', 'QASGX', 'HISFX']
    us_treasuries = ['^TNX', '^IRX', '^TYX']
    """
    yahoo_financials = YahooFinancials(tickers)
    historical_stock_prices = yahoo_financials.get_historical_price_data(start, end, 'daily')

    daily = pd.date_range(start, end, freq='D')
    df = pd.DataFrame(daily,columns=['date'])#['2'] = np.ones([393,1])

    for name in tickers:
        current = historical_stock_prices[name]['prices']
        df2 = pd.DataFrame(np.asarray([d['formatted_date'] for d in current]), columns=["date"])
        c_open = np.nan_to_num(np.asarray([x['open'] if x['open'] is not None else 0.1 for x in current]))
        c_close = np.nan_to_num(np.asarray([x['close'] if x['close'] is not None else 0.1 for x in current]))

        df2[str(name) + "%"] = ((c_close / c_open) -1) * 100
        df2[str(name) + "_Y"] = df2.apply(lambda row: classificator(row, str(name) + "%"),axis=1)
        df2[str(name) + "_Vol"] = np.asarray([x['volume'] for x in current])
        df2[str(name) + "_YPred"] = np.append(df2.apply(lambda row: classificator(row, str(name) + "%"),axis=1)[1:],[None])
        df = df.set_index('date').join(df2.set_index('date')).reset_index()
    return df.set_index('date')